# Azure AI Search Query Patterns
Azure OpenAI 와 Azure AI Search Python client library 기반의 Azure AI Search 다양한 검색 방식에 대한 Code Sample 입니다. forked from [https://github.com/Azure/azure-search-vector-samples]

## Prerequisites
3.10 또는 이후 버전의 Python 가상 환경을 구성합니다 : 
 1. 명령 Palette를 엽니다 (Ctrl+Shift+P).
 1. Python을 검색합니다: Create Environment를 선택합니다.
 1. Venv / Conda를 선택하고 새 환경을 만들 위치를 선택합니다.
 1. Python interpreter 버전을 선택합니다. 3.10 이상의 버전으로 생성합니다.

디펜던시 설치를 위해 아래 코드를 실행하여 실행에 필요한 package들을 설치하세요. 

```bash
pip install -r requirements.txt
```

## Set up your environment
현재의 repository를 Clone 하세요. 

```bash
git clone https://github.com/hyogrin/Azure_OpenAI_samples.git
```

.env-sample 파일을 기반으로 .env 파일을 만듭니다. 새 .env 파일을 노트북이 들어 있는 폴더에 복사하고 변수를 업데이트합니다.

## Import required libraries and environment variables

In [19]:
import os
import json
from openai import AzureOpenAI
import sys
import pandas as pd
import tiktoken

from dotenv import load_dotenv

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.models import VectorizableTextQuery
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchField,
    SearchFieldDataType,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    AzureOpenAIVectorizer,
    AzureOpenAIParameters,
    SemanticConfiguration,
    SemanticSearch,
    SemanticPrioritizedFields,
    SemanticField,
    SearchIndex
)

from tenacity import retry, wait_random_exponential, stop_after_attempt

In [20]:
sys.version

'3.11.10 (main, Oct  3 2024, 07:29:13) [GCC 11.2.0]'

In [21]:
# Configure environment variables
load_dotenv()
index_name = "quickdemo"
MAX_RETRIES = 3
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
key = os.getenv("AZURE_SEARCH_API_KEY")


client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),        # Azure OpenAI base URL
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("AZURE_OPENAI_API_VERSION"),
    max_retries    = MAX_RETRIES
)

deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT")
deployment_embedding_name = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")

credential = AzureKeyCredential(key)

## Create embeddings
- 데이터를 읽고, OpenAI 임베딩을 생성하고, 유효한 형식으로 내보내서 Azure AI Search 인덱스를 삽입합니다

In [22]:
!ls text-sample.json -lh

ls: cannot access 'text-sample.json': No such file or directory


In [23]:
# Generate Document Embeddings using OpenAI Ada 002

# Read the text-sample.json
# with open("text-sample.json", "r", encoding="utf-8") as file:
#     input_data = json.load(file)

df_input_data = pd.read_json(os.path.join(os.getcwd(),'data/text-sample.json'))    

In [24]:
#pd.options.mode.chained_assignment = None #https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#evaluation-order-matters

# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

df_input_data['content']= df_input_data["content"].apply(lambda x : normalize_text(x))

- Azure OpenAI에서 제공하는 Embedding API를 활용하기 위해 문서에서 Text 길이가 8,192 토큰이 넘지 않는 문서를 확인합니다

In [25]:
tokenizer = tiktoken.get_encoding("cl100k_base")
df_input_data['n_tokens'] = df_input_data["content"].apply(lambda x: len(tokenizer.encode(x)))
df_input_data = df_input_data[df_input_data.n_tokens<8192]
len(df_input_data)
df_input_data

,id,title,content,category,n_tokens
0,1,Azure App Service,Azure App Service is a fully managed platform ...,Web,93
1,2,Azure Functions,Azure Functions is a serverless compute servic...,Compute,85
2,3,Azure Cognitive Services,Azure Cognitive Services are a set of AI servi...,AI + Machine Learning,91
3,4,Azure Storage,"Azure Storage is a scalable, durable, and high...",Storage,95
4,5,Azure SQL Database,Azure SQL Database is a fully managed relation...,Databases,93
...,...,...,...,...,...
103,104,Azure Site Recovery,"Azure Site Recovery is a fully managed, disast...",Management + Governance,103
104,105,Azure Web PubSub,"Azure Web PubSub is a fully managed, real-time...",Web,100
105,106,Azure Data Factory,Azure Data Factory is a cloud-based data integ...,Analytics,108
106,107,Azure Data Bricks,"Azure Data Bricks is a fully managed, Apache S...",Analytics,110


- 확인을 완료 한 뒤 필요 없어진 column을 삭제하고, Azure AI Search에 삽입하기 위해 데이터를 저장합니다. 

In [26]:
df_input_data = df_input_data.drop('n_tokens', axis=1)

In [27]:
# Generate Document Embeddings using OpenAI Ada 002
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# Function to generate embeddings for title and content fields, also used for query embeddings
def get_embedding(text, model=deployment_embedding_name): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [28]:
# model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
df_input_data['titleVector'] = df_input_data["title"].apply(lambda x : get_embedding (x, model = deployment_embedding_name)) 
df_input_data['contentVector'] = df_input_data["content"].apply(lambda x : get_embedding (x, model = deployment_embedding_name)) 
df_input_data

,id,title,content,category,titleVector,contentVector
0,1,Azure App Service,Azure App Service is a fully managed platform ...,Web,"[-0.010541444644331932, -0.02157960645854473, ...","[0.007762514520436525, -0.023638280108571053, ..."
1,2,Azure Functions,Azure Functions is a serverless compute servic...,Compute,"[-0.010190718807280064, -0.005175409838557243,...","[-0.014527262188494205, -0.012572179548442364,..."
2,3,Azure Cognitive Services,Azure Cognitive Services are a set of AI servi...,AI + Machine Learning,"[-0.012722955085337162, -0.004301734268665314,...","[-0.016163231804966927, -0.0014309135731309652..."
3,4,Azure Storage,"Azure Storage is a scalable, durable, and high...",Storage,"[-0.02305583469569683, -0.014231489039957523, ...","[-0.01971307024359703, -0.014362750574946404, ..."
4,5,Azure SQL Database,Azure SQL Database is a fully managed relation...,Databases,"[-0.010304322466254234, -0.00812271423637867, ...","[-0.0061157881282269955, -0.011781413108110428..."
...,...,...,...,...,...,...
103,104,Azure Site Recovery,"Azure Site Recovery is a fully managed, disast...",Management + Governance,"[-0.0063414908945560455, -0.007637436036020517...","[-0.00913618691265583, -0.018338577821850777, ..."
104,105,Azure Web PubSub,"Azure Web PubSub is a fully managed, real-time...",Web,"[-0.01065534446388483, -0.009444351308047771, ...","[-0.006847419310361147, -0.007372125517576933,..."
105,106,Azure Data Factory,Azure Data Factory is a cloud-based data integ...,Analytics,"[-0.01965930685400963, -0.01779160276055336, -...","[-0.028729742392897606, -0.021181656047701836,..."
106,107,Azure Data Bricks,"Azure Data Bricks is a fully managed, Apache S...",Analytics,"[-0.011949058622121811, -0.014182858169078827,...","[-0.030274879187345505, -0.006946539506316185,..."


In [29]:
df_input_data.to_csv(os.path.join(os.getcwd(),'data/embedding_input_data.csv'), index=False)

## Create your search index
Create your search index schema and vector search configuration:

In [42]:
# Create a search index
index_client = SearchIndexClient(endpoint=search_endpoint, credential=credential)
fields = [
    SearchField(name="id", key=True, type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=False),  
    SearchField(name="title", type=SearchFieldDataType.String, searchable=True),
    SearchField(name="content", type=SearchFieldDataType.String, searchable=True),
    SearchField(name="content_ko", type=SearchFieldDataType.String, searchable=True, analyzer_name="ko.lucene"),
    SearchField(name="category", type=SearchFieldDataType.String, filterable=True),
    SearchField(
        name="titleVector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=1536,
        vector_search_profile_name="myHnswProfile",  # Ensure vector_search_profile is set
    ),
    SearchField(
        name="contentVector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=1536,
        vector_search_profile_name="myHnswProfile",  # Ensure vector_search_profile is set
    ),
]
# Configure the vector search configuration  
vector_search = VectorSearch(  
    algorithms=[  
        HnswAlgorithmConfiguration(name="myHnsw"),
    ],  
    profiles=[  
        VectorSearchProfile(  
            name="myHnswProfile",  
            algorithm_configuration_name="myHnsw",  
            vectorizer="myOpenAI",  
        )
    ],  
    vectorizers=[  
        AzureOpenAIVectorizer(  
            name="myOpenAI",  
            kind="azureOpenAI",  
            azure_open_ai_parameters=AzureOpenAIParameters(  
                resource_uri=os.getenv("AZURE_OPENAI_ENDPOINT"),  
                deployment_id=deployment_embedding_name,
                model_name=deployment_embedding_name,
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            ),
        ),  
    ],  
)  


  
semantic_config = SemanticConfiguration(  
    name="my-semantic-config",  
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="title"),
        content_fields=[SemanticField(field_name="content")]  
    ),  
)

# Create the semantic search with the configuration  
semantic_search = SemanticSearch(configurations=[semantic_config])  
  
# Create the search index
index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search, semantic_search=semantic_search)  
result = index_client.create_or_update_index(index)  
print(f"{result.name} created")  

quickdemo created


## Insert text and embeddings into vector store
Add texts and metadata from the JSON data to the vector store:

In [43]:
# Upload data_embeddings.csv to search_client.upload_documents to create the index 
ddocuments = df_input_data.to_dict(orient='records')
for doc in documents:
    doc['id'] = str(doc['id'])
search_client = SearchClient(
    endpoint=search_endpoint, index_name=index_name, credential=credential
)
result = search_client.upload_documents(documents)

print(f"Uploaded {len(documents)} documents")

Uploaded 108 documents


## Perform a vector similarity search

- 가장 basic한 vector similarity search를 수행합니다.

In [44]:
from azure.search.documents.models import VectorizedQuery

# Pure Vector Search
query = "I want to monitor my applications"


search_client = SearchClient(search_endpoint, index_name, credential=credential)
embedding = get_embedding (query, model = deployment_embedding_name)

vector_query = VectorizedQuery(vector=embedding, k_nearest_neighbors=3, fields="contentVector")


results = search_client.search(
    search_text=None,
    vector_queries=[vector_query],
    select=["title", "content", "category"],
)

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}\n")

Title: Azure Application Insights
Score: 0.84255695
Content: Azure Application Insights is an application performance management service that enables you to monitor, diagnose, and troubleshoot your applications and infrastructure. It provides features like real-time telemetry, dependency mapping, and live metrics. Application Insights supports various platforms, such as .NET, Java, Node.js, and Python. You can use Azure Application Insights to detect and diagnose issues, optimize your performance, and ensure the availability of your applications. It also integrates with other Azure services, such as Azure Monitor and Azure Log Analytics.
Category: Management + Governance

Title: Azure Monitor
Score: 0.8420792
Content: Azure Monitor is a comprehensive, full-stack monitoring service that enables you to collect, analyze, and act on telemetry data from your applications and infrastructure. It provides features like log analytics, metrics, alerts, and dashboards. Azure Monitor supports vari

- Azure AI Search에서 제공하는 VectorizableTextQuery 클래스를 활용하여 query를 별도로 embedding후 조회하지 않고 곧바로 유사한 문서를 찾습니다.
- 다른 언어로 조회하는 경우에도 동일하게 사용할 수 있습니다.

In [45]:
# Pure Vector Search multi-lingual (e.g 'tools for software development' in Dutch)
query = "Application을 모니터링하고 싶어"

search_client = SearchClient(search_endpoint, index_name, credential=credential)
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=3, fields="contentVector")

results = search_client.search(
    search_text=None,
    vector_queries=[vector_query],
    select=["title", "content", "category"],
)

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}\n")

Title: Azure Application Insights
Score: 0.8036648
Content: Azure Application Insights is an application performance management service that enables you to monitor, diagnose, and troubleshoot your applications and infrastructure. It provides features like real-time telemetry, dependency mapping, and live metrics. Application Insights supports various platforms, such as .NET, Java, Node.js, and Python. You can use Azure Application Insights to detect and diagnose issues, optimize your performance, and ensure the availability of your applications. It also integrates with other Azure services, such as Azure Monitor and Azure Log Analytics.
Category: Management + Governance

Title: Azure Monitor
Score: 0.8024742
Content: Azure Monitor is a comprehensive, full-stack monitoring service that enables you to collect, analyze, and act on telemetry data from your applications and infrastructure. It provides features like log analytics, metrics, alerts, and dashboards. Azure Monitor supports vario

- 인덱스설정을 무시하고 벡터 인덱스를 완전검색(exhaustive KNN)으로 조회하는 방법입니다. 필터와 함께 조합하면 좋습니다. 튜닝의 기준점이 되는 ground-truth값을 체크할 수 있습니다. 

In [46]:
# Pure Vector Search
query = "tools for software development"  
  
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=3, fields="contentVector", exhaustive=True)
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["title", "content", "category"],
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  

Title: Azure DevOps
Score: 0.8290269
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.81771946
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports various platforms, such

## Cross-Field Vector Search

- ContentVector외에 추가적으로 titleVector를 사용하여 Cross-Field Vector Search를 수행할 수 있습니다.

In [47]:
# Cross-Field Vector Search
query = "I want to monitor my applications"

search_client = SearchClient(search_endpoint, index_name, credential=credential)
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=3, fields="titleVector, contentVector")

results = search_client.search(
    search_text=None,
    vector_queries=[vector_query],
    select=["title", "content", "category"],
)

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}\n")

Title: Azure Monitor
Score: 0.03306011110544205
Content: Azure Monitor is a comprehensive, full-stack monitoring service that enables you to collect, analyze, and act on telemetry data from your applications and infrastructure. It provides features like log analytics, metrics, alerts, and dashboards. Azure Monitor supports various data sources, such as Azure resources, custom applications, and operating systems. You can use Azure Monitor to detect and diagnose issues, optimize your performance, and ensure the availability of your resources. It also integrates with other Azure services, such as Azure Log Analytics and Azure Application Insights.
Category: Management + Governance

Title: Azure Application Insights
Score: 0.03306011110544205
Content: Azure Application Insights is an application performance management service that enables you to monitor, diagnose, and troubleshoot your applications and infrastructure. It provides features like real-time telemetry, dependency mapping, and l

## Perform a Multi-Vector Search

- 하나의 쿼리를 두개의 VectorField에 사용하거나 하나의 VectorField에 두가지 쿼리를 사용하여 Multi-Vector Search를 수행할 수 있습니다.
- 예를 들어, query rewrite를 통해 추가적인 query를 생성하여 Multi-Vector Search를 수행할 수 있습니다.
- 이때 가중치를 부여하여 결과를 조정할 수 있습니다.

In [48]:
# Multi-Vector Search
query1 = "software development for ds"
query2 = "데이터를 가지고 실험을 하거나 통계를 보고 모델을 제작할 때 사용할 수 있는 도구"

search_client = SearchClient(search_endpoint, index_name, credential=credential)
vector_query1 = VectorizableTextQuery(text=query1, k_nearest_neighbors=3, fields="titleVector, contentVector", weight=2)
vector_query2 = VectorizableTextQuery(text=query2, k_nearest_neighbors=3, fields="titleVector, contentVector", weight=0.5)
# vector_query1 = VectorizableTextQuery(text=query1, k_nearest_neighbors=3, fields="titleVector, contentVector", weight=2)
# vector_query2 = VectorizableTextQuery(text=query2, k_nearest_neighbors=3, fields="titleVector, contentVector", weight=0.5)



results = search_client.search(
    search_text=None,
    vector_queries=[vector_query1, vector_query2],
    select=["title", "content", "category"],
)

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}\n")

Title: Azure DevTest Labs
Score: 0.0416666679084301
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports various platforms, such as Windows, Linux, and Kubernetes. You can use Azure DevTest Labs to improve your application development lifecycle, reduce your costs, and ensure the consistency of your environments. It also integrates with other Azure services, such as Azure Virtual Machines and Azure App Service.
Category: Developer Tools

Title: Azure Databricks
Score: 0.04139785096049309
Content: Azure Databricks is an Apache Spark-based analytics platform optimized for the Azure cloud. It provides a collaborative workspace for data scientists, engineers, and business users to process, analyze, and visualize big data. Databricks supports multiple programming languages, including P

## Perform a Pure Vector Search with a filter

- Vector Search에 필터를 추가하여 결과를 제한할 수 있습니다 특히 검색 범위가 넓고 데이터량이 많은 경우에 유용합니다. 

In [49]:
from azure.search.documents.models import VectorFilterMode

# Pure Vector Search with Filter
query = "tools for software development"

search_client = SearchClient(search_endpoint, index_name, credential=credential)
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=3, fields="contentVector")

results = search_client.search(
    search_text=None,
    vector_queries=[vector_query],
    vector_filter_mode=VectorFilterMode.PRE_FILTER,
    filter="category eq 'Developer Tools'",
    select=["title", "content", "category"]
)

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}\n")

Title: Azure DevOps
Score: 0.8290269
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.81771946
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports various platforms, such

## Perform a Hybrid Search

- searchtext와 vector search를 혼합하여 검색을 수행할 수 있습니다.

In [50]:
# Hybrid Search
query = "integration"

search_client = SearchClient(search_endpoint, index_name, AzureKeyCredential(key))
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=3, fields="contentVector")

vector_query_result = search_client.search(
    search_text=None, vector_queries=[vector_query], select=["title", "content", "category"], top=3)

hybrid_query_result = search_client.search(
    search_text=query, vector_queries=[vector_query], select=["title", "content", "category"], top=3)

print("----------------------------vector search---------------------------------")

for result in vector_query_result:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}\n")

print("----------------------------hybrid search---------------------------------")

for result in hybrid_query_result:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}\n")    

----------------------------vector search---------------------------------


Title: Azure Logic Apps
Score: 0.80287653
Content: Azure Logic Apps is a cloud-based service that allows you to create and run workflows that integrate with various services and data sources. It provides a visual designer for creating workflows, and supports a wide range of connectors for popular services like Office 365, Salesforce, and Dropbox. Logic Apps enable you to automate tasks, process and transform data, and orchestrate complex processes without writing code. You can monitor and manage your workflows using Azure Portal, REST APIs, or PowerShell.
Category: Integration

Title: Azure Synapse Analytics
Score: 0.80188507
Content: Azure Synapse Analytics is an integrated analytics service that brings together big data and data warehousing. It enables you to ingest, prepare, manage, and serve data for immediate business intelligence and machine learning needs. Synapse Analytics provides a unified workspace for data engineers, data scientists, and business analysts to collaborate and

- vector query에 대한 가중치를 부여하여 결과를 조정할 수 있습니다.

In [51]:
# Hybrid Search
query = "integration"

search_client = SearchClient(search_endpoint, index_name, AzureKeyCredential(key))
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=3, fields="contentVector", weight=0.2)

vector_query_result = search_client.search(
    search_text=None, vector_queries=[vector_query], select=["title", "content", "category"], top=3)

hybrid_query_result = search_client.search(
    search_text=query, vector_queries=[vector_query], select=["title", "content", "category"], top=3)

print("----------------------------vector search---------------------------------")

for result in vector_query_result:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}\n")

print("----------------------------hybrid search---------------------------------")

for result in hybrid_query_result:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}\n")    

----------------------------vector search---------------------------------


Title: Azure Logic Apps
Score: 0.80287653
Content: Azure Logic Apps is a cloud-based service that allows you to create and run workflows that integrate with various services and data sources. It provides a visual designer for creating workflows, and supports a wide range of connectors for popular services like Office 365, Salesforce, and Dropbox. Logic Apps enable you to automate tasks, process and transform data, and orchestrate complex processes without writing code. You can monitor and manage your workflows using Azure Portal, REST APIs, or PowerShell.
Category: Integration

Title: Azure Synapse Analytics
Score: 0.80188507
Content: Azure Synapse Analytics is an integrated analytics service that brings together big data and data warehousing. It enables you to ingest, prepare, manage, and serve data for immediate business intelligence and machine learning needs. Synapse Analytics provides a unified workspace for data engineers, data scientists, and business analysts to collaborate and

## hybrid search + Semantic reranking

- hybrid search한 결과를  semantic reranking을 혼합하여 검색을 수행할 수 있습니다.

In [52]:
from azure.search.documents.models import QueryType, QueryCaptionType, QueryAnswerType

# Semantic Hybrid Search
query = "what is azure sarch?"

vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=3, fields="contentVector", exhaustive=True)

results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["title", "content", "category"],
    query_type=QueryType.SEMANTIC, semantic_configuration_name='my-semantic-config', query_caption=QueryCaptionType.EXTRACTIVE, query_answer=QueryAnswerType.EXTRACTIVE,
    top=3
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(f"Title: {result['title']}")
    print(f"Reranker Score: {result['@search.reranker_score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")


Semantic Answer: Azure Cognitive Search. Azure Cognitive Search is<em> a fully managed search-as-a-service that enables you to build rich search experiences for your applications.</em> It provides features like full-text search, faceted navigation, and filters. Azure Cognitive Search supports various data sources, such as Azure SQL Database, Azure Blob Storage, and Azure Cosmos DB. You can use Azure Cognitive Search to index your data, create custom scoring profiles, and integrate with other Azure services. It also integrates with other Azure services, such as Azure Cognitive Services and Azure Machine Learning..
Semantic Answer Score: 0.92919921875

Title: Azure Stack Edge
Reranker Score: 2.3630130290985107
Content: Azure Stack Edge is a managed, edge computing appliance that enables you to run Azure services and AI workloads on-premises or at the edge. It provides features like hardware-accelerated machine learning, local caching, and integration with Azure IoT Hub. Azure Stack Edge 